<a href="https://colab.research.google.com/github/pirsquared/Advent-of-Code/blob/master/2018/Day11_improved.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Faster
`numba` isn't necessary but does help

In [1]:
!pip install numba

In [0]:
import pandas as pd
import numpy as np

from numba import njit

In [0]:
 def get_a(data):
    d = int(data)
    j = np.arange(1, 301)
    i = np.arange(1, 301)
    r = j + 10
    p = (r * i[:, None] + d) * r // 100 % 10 - 5
    return p

In [0]:
@njit
def f(c):
  n = c.shape[0]
  n = n - 1
  maxpower = np.array([0, 0, 0, 0])
  
  for i in range(n):
    for j in range(n):
      for k in range(n - max(i, j)):
        power = g(c, i, j, k)
        if power > maxpower[-1]:
          maxpower[:] = [j + 1, i + 1, k, power]
          
  return maxpower
        
@njit
def g(a, i, j, k):
  return a[i+k, j+k] - a[i, j+k] - a[i+k, j] + a[i, j]
        

# Deal Maker
By calculating the 2-D cumulative sumation, I've pre-calculated much of what will be needed to determine the convolutions.  I'll will have reduced every sum over a kernel to an operation over four numbers.

In [0]:
def pad(a):
  n, m = a.shape
  c = np.zeros((n + 1, m + 1), a.dtype)
  c[1:, 1:] = a.cumsum(0).cumsum(1)
  
  return c

In [0]:
def part2(data):
  *a, p = f(pad(get_a(int(data))))
  return f"""Power: {p}
  AoC: {','.join(map(str, a))}"""

In [7]:
print(part2(18))

Power: 113
  AoC: 90,269,16


In [8]:
print(part2(42))

Power: 119
  AoC: 232,251,12


In [9]:
print(part2(5177))

Power: 80
  AoC: 231,135,8


In [10]:
%timeit part2(5177)

10 loops, best of 3: 20.7 ms per loop


# Trying to Explain

In [0]:
import pandas as pd

In [0]:
a = get_a(18)

In [0]:
df = pd.DataFrame(a)
df_cumsum = df.cumsum(0).cumsum(1)
df_subset = df.loc[267:284, 88:105]
df_cumsum_subset = df_cumsum.loc[267:284, 88:105]

This is the 16x16 grid whos upper left corner starts at 269, 90

In [14]:
def highlight_cols(s):
    color = 'yellow'
    return f'background-color: {color}'

interior = pd.IndexSlice[268:283, 89:104]
df_subset.style.applymap(highlight_cols, subset=interior)

,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105
267,-1,3,1,-4,-5,0,0,-4,-3,3,-5,2,-5,3,-4,4,-2,-3
268,-3,3,3,0,1,-2,1,-2,2,0,4,3,-2,-2,3,4,0,2
269,-5,3,-5,4,-3,-4,1,1,-4,-3,3,4,1,3,1,4,3,-3
270,3,3,-3,-2,3,-5,1,3,1,3,2,-5,4,-1,-1,4,-5,1
271,1,3,-1,2,0,3,1,-4,-5,0,0,-4,-2,4,-3,4,-3,-4
272,-1,3,1,-4,-4,1,2,-2,0,-3,-1,-3,1,0,4,4,-1,0
273,-3,3,3,0,2,-1,2,0,4,3,-2,-2,4,-5,2,4,2,-5
274,-5,3,-5,4,-2,-3,2,3,-1,0,-3,-1,-3,1,0,4,4,-1
275,3,3,-3,-2,4,-5,2,-5,3,-3,-4,0,0,-4,-3,4,-4,4
276,1,3,-1,2,0,4,3,-3,-3,3,4,1,4,2,-5,4,-2,-1


We can see the sum accross that array is the power we expect

In [15]:
df_subset.loc[268:283, 89:104].values.sum()

113

But instead if we calculate the 2-D cumulative sum. we can reference specific points to calculate the same as the sum the the array above.

- **Yellow** are the cumsums at same location as the yellow in the dataframe above.
- **Cyan** are the corners I will reference in the formula to calc the same sumation.

In [16]:
def highlight_cols(s):
    color = 'yellow'
    return f'background-color: {color}'
  
def highlight_cell(s):
    color = 'cyan'
    return f'background-color: {color}'

interior = pd.IndexSlice[268:283, 89:104]
four_pts = pd.IndexSlice[[267, 283], [88, 104]]
df_cumsum_subset.style \
    .applymap(highlight_cols, subset=interior) \
    .applymap(highlight_cell, subset=four_pts)


,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105
267,-11173,-10369,-10539,-10683,-10828,-10964,-11128,-11265,-11398,-11521,-11650,-11792,-11931,-12076,-12213,-12383,-12516,-12659
268,-11247,-10440,-10607,-10751,-10895,-11033,-11196,-11335,-11466,-11589,-11714,-11853,-11994,-12141,-12275,-12441,-12574,-12715
269,-11318,-10508,-10680,-10820,-10967,-11109,-11271,-11409,-11544,-11670,-11792,-11927,-12067,-12211,-12344,-12506,-12636,-12780
270,-11344,-10531,-10706,-10848,-10992,-11139,-11300,-11435,-11569,-11692,-11812,-11952,-12088,-12233,-12367,-12525,-12660,-12803
271,-11421,-10605,-10781,-10921,-11065,-11209,-11369,-11508,-11647,-11770,-11890,-12034,-12172,-12313,-12450,-12604,-12742,-12889
272,-11440,-10621,-10796,-10940,-11088,-11231,-11389,-11530,-11669,-11795,-11916,-12063,-12200,-12341,-12474,-12624,-12763,-12910
273,-11462,-10640,-10812,-10956,-11102,-11246,-11402,-11543,-11678,-11801,-11924,-12073,-12206,-12352,-12483,-12629,-12766,-12918
274,-11516,-10691,-10868,-11008,-11156,-11303,-11457,-11595,-11731,-11854,-11980,-12130,-12266,-12411,-12542,-12684,-12817,-12970
275,-11504,-10676,-10856,-10998,-11142,-11294,-11446,-11589,-11722,-11848,-11978,-12128,-12264,-12413,-12547,-12685,-12822,-12971
276,-11540,-10709,-10890,-11030,-11174,-11322,-11471,-11617,-11753,-11876,-12002,-12151,-12283,-12430,-12569,-12703,-12842,-12992


In [17]:
df_cumsum.loc[four_pts]

,88,104
267,-11173,-12516
283,-11841,-13071


In [18]:
(ii, ij), (ji, jj) = df_cumsum.loc[four_pts].values
jj - ij - ji + ii

113

The point is that `jj - ij - ji + ii` is constant time while `df_subset.loc[268:283, 89:104].values.sum()` is O(k^2) time.


_____
# Another way
Slicing with numpy

In [0]:
def part2_1(data):
    c = pad(get_a(int(data)))

    i, j, k = np.indices((300, 300, 300)).reshape(3, -1)
    m = k < 300 - np.maximum(i, j)

    i = i[m]
    j = j[m]
    k = k[m]

    pos = (c[i, j] + c[i + k, j + k] - c[i, j + k] - c[i + k, j]).argmax()

    return ','.join(map(str, (j[pos] + 1, i[pos] + 1, k[pos])))

In [20]:
print(part2_1(18))

90,269,16


In [21]:
%timeit(part2_1(18))

1 loop, best of 3: 995 ms per loop
